In [12]:
# %pip install scikit-learn
# %pip install pandas

In [19]:
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
import pandas as pd
import requests
import json
import re

In [29]:
cosine_similarity([[3,1]],[[6,2]])
cosine_distances([[3,1]],[[6,2]])

array([[1.11022302e-16]])

In [ ]:
cosine_similarity([[3,0]],[[0,8]])

In [ ]:
cosine_similarity([[3,1]],[[3,2]])

In [14]:
doc1 = """
iphone sales contributed to 70% of revenue. iphone demand is increasing by 20% yoy. 
the main competitor phone galaxy recorded 5% less growth compared to iphone" ipad ipad 
"""

doc2 = """
The upside pressure on volumes for the iPhone 12 series, historical outperformance 
in the July-September time period heading into launch event, and further catalysts in relation
to outperformance for iPhone 13 volumes relative to lowered investor expectations implies a 
very attractive set up for the shares.
"""

doc3 = """
samsung's flagship product galaxy is able to penetrate more into asian markets compared to
iphone. galaxy is redesigned with new look that appeals young demographics. 60% of samsung revenues
are coming from galaxy phone sales
"""

doc4 = """
Samsung Electronics unveils its Galaxy S21 flagship, with modest spec improvements 
and a significantly lower price point. Galaxy S21 price is lower by ~20% (much like the iPhone 12A), 
which highlights Samsung's focus on boosting shipments and regaining market share.
"""

In [78]:
df = pd.DataFrame([
        {'iPhone': 3,'galaxy': 1, 'ipad': 4, 'hi': 0},
        {'iPhone': 2,'galaxy': 0, 'ipad': 1, 'hi': 0},
        {'iPhone': 1,'galaxy': 3, 'ipad': 0, 'hi': 0},
        {'iPhone': 1,'galaxy': 2, 'ipad': 2, 'hi': 0},
        {'iPhone': 0,'galaxy': 0, 'ipad': 0, 'hi': 0},
        {'iPhone': 0,'galaxy': 0, 'ipad': 0, 'hi': 0},
        {'iPhone': 0,'galaxy': 0, 'ipad': 0, 'hi': 0},
        {'iPhone': 0,'galaxy': 0, 'ipad': 0, 'hi': 0},
    ],
    index=[
        "doc1",
        "doc2",
        "doc3",
        "doc4",
        "doc5","doc6","doc7","doc8"
    ])

In [80]:
df
#df.loc["doc1":"doc1"]

,iPhone,galaxy,ipad,hi
doc1,3,1,4,0
doc2,2,0,1,0
doc3,1,3,0,0
doc4,1,2,2,0
doc5,0,0,0,0
doc6,0,0,0,0
doc7,0,0,0,0
doc8,0,0,0,0


In [76]:
cosine_similarity(df.loc["doc1":"doc1"],df.loc["doc5":"doc5"])

array([[0.]])

In [28]:
data =  ["independent","abruptness","abandonment"]
response_API = requests.post("http://127.0.0.1:5000/searchdata", json = {"word": "carvet"})

sendData = response_API.text
print(sendData)

{
  "carver": 1, 
  "carvet": 1, 
  "saigon": 2, 
  "sargon": 3
}



In [98]:
def getTableIndex(listBooksData):
    # counts = dict()
    # word2letters = dict()
    # word3letters = dict()
    tableIndex = dict()
    booksInfo = []

    for book in listBooksData:
        response_API = requests.get(book['text_url'])
        data = response_API.text
        #### Option 1: Prendre seulement des mots avec carateres de 4 à 10
        words = re.findall(r"[A-Za-z]{4,10}\w+", data)
        occurentCounts = dict()

        for word in words:
            w = word.lower()

            # Count for table index all books
            if w in tableIndex:
                if book['id'] in tableIndex[w]:
                    tableIndex[w][book['id']] += 1
                else:
                    tableIndex[w][book['id']] = 1
            else:
                tableIndex[w] = dict({book['id']: 1})

            # Count for table index for each book
            if w in occurentCounts:
                occurentCounts[w] += 1
            else:
                occurentCounts[w] = 1

        booksInfo.append({
            "bookId": book['id'],
            "words": occurentCounts,
            "totalWords": len(words),
            "totalWordsWithOccur": len(occurentCounts)
        })  
    return tableIndex, booksInfo

# request to get books
def getBooksData(listBooks):
    booksData = []
    for book in listBooks:
        response_API = requests.get('https://gutendex.com/books/{}'.format(book))
        #print(response_API.status_code)
        data = response_API.text
        parse_json = json.loads(data)
        booksData.append(parse_json)
    return booksData

def getListBooks(listBooks):
    data = getBooksData(listBooks)
    result = []
    for d in data:
        for t in d['formats'].keys():
            checkEnd = d['formats'][t].split('.').pop()
            if(checkEnd == 'txt'):
                result.append({
                    'id': d['id'],
                    'text_url': d['formats'][t]
                })
    return result


In [99]:

listBooks = [49345,56667,1,2,3,4,5,6,7]
listBooksData = getListBooks(listBooks)
tableIndexData, booksInfo = getTableIndex(listBooksData)
historyWords =dict({
  "carver": 1, 
  "carvet": 1, 
  "saigon": 2, 
  "sargon": 3
})


In [137]:

def cosineSimilarity(historyWords, tableIndexData):
    booksData = dict({'history':historyWords})
    # booksIndex = ['history']

    for word in historyWords.keys():
        if word in tableIndexData:
            print(tableIndexData[word])
            for b in tableIndexData[word].keys(): 
                if b in booksData:
                    # print(dict({word:tableIndexData[word][b]}))
                    booksData[b].update(dict({word:tableIndexData[word][b]}))
                else:
                    booksData.update(dict({b: dict({word:tableIndexData[word][b]})}))
                
    print("Show resuft")
    [print("{} - {}".format(b, booksData[b])) for b in booksData]

    bookDF = pd.DataFrame(booksData.values(),
        index=booksData.keys()).fillna(0)

    print(bookDF)
    

    # for book in booksInfo:
    #     count = 0
    #     booksIndex.append(book['bookId'])
    #     booksWords = dict()
    #     for w in historyWords.keys():
    #         if w in book['words']:
    #             booksWords[w] = book['words'][w]
    #         else:
    #             booksWords[w] = 0
    #     booksData.append(booksWords)

    # bookDF = pd.DataFrame(booksData,
    #     index=booksIndex)

    # print(bookDF)

    # for cs in booksIndex[1:]:
    #     res = cosine_similarity(bookDF.loc["history":"history"],bookDF.loc[cs:cs])
    #     print(res)

cosineSimilarity(historyWords, tableIndexData)


{1: 1, 7: 1}
{49345: 1}
{49345: 98, 56667: 27}
Show resuft
history - {'carver': 1, 'carvet': 1, 'saigon': 2, 'sargon': 3}
1 - {'carver': 1}
7 - {'carver': 1}
49345 - {'saigon': 1, 'sargon': 98}
56667 - {'sargon': 27}
         carver  carvet  saigon  sargon
history     1.0     1.0     2.0     3.0
1           1.0     NaN     NaN     NaN
7           1.0     NaN     NaN     NaN
49345       NaN     NaN     1.0    98.0
56667       NaN     NaN     NaN    27.0


In [94]:
booksData = []
for book in booksInfo:
    count = 0
    for w in historyWords.keys():
        if w in book['words']:
            # count number occurs in document
            # count += book['words'][w]
            # count if have that word in ducument
            count += historyWords[w]
    booksData.append({
        "bookId": book['bookId'],
        "jaccard": (count/book['totalWordsWithOccur'])
    })

[print(b) for b in booksData]

{'bookId': 49345, 'jaccard': 0.0007574609907589759}
{'bookId': 56667, 'jaccard': 0.0004023605150214592}
{'bookId': 1, 'jaccard': 0.0003757985719654265}
{'bookId': 2, 'jaccard': 0.0}
{'bookId': 3, 'jaccard': 0.0}
{'bookId': 4, 'jaccard': 0.0}
{'bookId': 5, 'jaccard': 0.0}
{'bookId': 6, 'jaccard': 0.0}
{'bookId': 7, 'jaccard': 0.0023923444976076554}


[None, None, None, None, None, None, None, None, None]